In [1]:
import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError
import pandas as pd

USERNAME = 'basslaughter' #your spotify username
CLIENT_ID = '63594c9b2f99411a8cbd18df04851fc4' #set at your developer account
CLIENT_SECRET = '096168b2bd1f4378ae410726955c9ed8' #set at your developer account
REDIRECT_URI = 'http://google.com/' #set at your developer account, usually "http://localhost:8000"
SCOPE = 'user-library-read' # or else
# ps. REDIRECT_URI is crucial here. if http://localhost:8000 is not set, or with a single '/' misplaced, it will generate a connection error.

# then pass them:

token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

if token:
   sp = spotipy.Spotify(auth=token)

In [2]:
#TEST the API lookup

for k,v in sp.audio_features('54flyrjcdnQdco7300avMJ')[0].items():
    print(k,v)

danceability 0.693
energy 0.497
key 2
loudness -7.316
mode 1
speechiness 0.119
acousticness 0.679
instrumentalness 0
liveness 0.258
valence 0.473
tempo 81.308
type audio_features
id 54flyrjcdnQdco7300avMJ
uri spotify:track:54flyrjcdnQdco7300avMJ
track_href https://api.spotify.com/v1/tracks/54flyrjcdnQdco7300avMJ
analysis_url https://api.spotify.com/v1/audio-analysis/54flyrjcdnQdco7300avMJ
duration_ms 122067
time_signature 4


In [3]:
#Function to extract audio features with Spotify Track ID via Spotipy

def get_features(trackID):
        return sp.audio_features(trackID)[0]
    
get_features('54flyrjcdnQdco7300avMJ')

{'danceability': 0.693,
 'energy': 0.497,
 'key': 2,
 'loudness': -7.316,
 'mode': 1,
 'speechiness': 0.119,
 'acousticness': 0.679,
 'instrumentalness': 0,
 'liveness': 0.258,
 'valence': 0.473,
 'tempo': 81.308,
 'type': 'audio_features',
 'id': '54flyrjcdnQdco7300avMJ',
 'uri': 'spotify:track:54flyrjcdnQdco7300avMJ',
 'track_href': 'https://api.spotify.com/v1/tracks/54flyrjcdnQdco7300avMJ',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/54flyrjcdnQdco7300avMJ',
 'duration_ms': 122067,
 'time_signature': 4}

In [6]:
#Function to retrieve Spotify Track ID from Artist / Song via Spotipy

def get_spotify_id(artist,track):
    results = sp.search(q='artist:' + artist + ' ' + 'track:' + track, type='track')
    return results["tracks"]["items"][0]["id"]

get_spotify_id("Queen","We Will Rock You")

'54flyrjcdnQdco7300avMJ'

In [7]:
song_list = pd.read_pickle("./song_list.pkl")

In [8]:
song_list.head(2)

,trackId,songId,artist,track
0,TRMMMYQ128F932D901,SOQMMHC12AB0180CB8,Faster Pussy cat,Silent Night
1,TRMMMKD128F425225D,SOVFVAK12A8C1350D9,Karkkiautomaatti,Tanssi vaan


In [9]:
song_list.head()

,trackId,songId,artist,track
0,TRMMMYQ128F932D901,SOQMMHC12AB0180CB8,Faster Pussy cat,Silent Night
1,TRMMMKD128F425225D,SOVFVAK12A8C1350D9,Karkkiautomaatti,Tanssi vaan
2,TRMMMRX128F93187D9,SOGTUKN12AB017F4F1,Hudson Mohawke,No One Could Ever
3,TRMMMCH128F425532C,SOBNYVR12A8C13558C,Yerba Brava,Si Vos Querés
4,TRMMMWA128F426B589,SOHSBXH12A8C13B0DF,Der Mystic,Tangle Of Aspens


In [10]:
song_list2 = song_list[["artist","track"]]

In [11]:
song_list2.head()

,artist,track
0,Faster Pussy cat,Silent Night
1,Karkkiautomaatti,Tanssi vaan
2,Hudson Mohawke,No One Could Ever
3,Yerba Brava,Si Vos Querés
4,Der Mystic,Tangle Of Aspens


In [12]:
import sqlite3

def create_table(name):
    conn = sqlite3.connect(f'./{name}.sqlite3')
    curs = conn.cursor()
    create_table = 'CREATE TABLE File (artist varchar(1000),track varchar(1000), danceability float, energy float, key float, loudness float, mode float, speechiness float, acousticness float, instrumentalness float, liveness float, valence float, tempo float, duration_ms float,time_signature float);' 
    curs.execute(create_table)
    return conn

def insert(conn,f):    
    curs = conn.cursor()    
    query = (f'INSERT INTO File (artist, track, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms,time_signature) VALUES (\"{f["artist"]}\",\"{f["track"]}\", {f["danceability"]}, {f["energy"]}, {f["key"]}, {f["loudness"]}, {f["mode"]}, {f["speechiness"]}, {f["acousticness"]},  {f["instrumentalness"]}, {f["liveness"]}, {f["valence"]}, {f["tempo"]}, {f["duration_ms"]},{f["time_signature"]});')
#     print(query)
    curs.execute(query)
    curs.close()
    conn.commit()
#     print('Inserted into DB: ', s['filename'])

In [223]:
# create_table("bronco")

In [225]:
conn = create_table("bronco")
insert(conn,feature_list[0])

In [218]:
# def retrieve_track_features(rownumbers):
#     feature_list = []
#     for entry,row in song_list2.head(rownumbers).iterrows():
#         f = {}
#         artist = row["artist"]
#         track = row["track"]
#         try:
#             spotifyid = get_spotify_id(artist,track)
#             features = get_features(spotifyid)
# #             f = {"artist":artist,"track":track,'danceability':features['danceability'],
# #                                  'energy':features['energy'],'key':features['key'],
# #                                  'loudness':features['loudness'],'mode':features['mode'],
# #                                 'speechiness':features['speechiness'],'acousticness':features['acousticness'],
# #                                  'instrumentalness':features['instrumentalness'],'liveness':features['liveness'],
# #                                  'valence':features['valence'],'tempo':features['tempo'],'duration_ms':features['duration_ms'],'time_signature':features['time_signature']}
#             feature_list.append({"artist":artist,"track":track,'danceability':features['danceability'],
#                                  'energy':features['energy'],'key':features['key'],
#                                  'loudness':features['loudness'],'mode':features['mode'],
#                                 'speechiness':features['speechiness'],'acousticness':features['acousticness'],
#                                  'instrumentalness':features['instrumentalness'],'liveness':features['liveness'],
#                                  'valence':features['valence'],'tempo':features['tempo'],'duration_ms':features['duration_ms'],'time_signature':features['time_signature']})
#         except:
#             continue
# #         return f
#     return feature_list


# # retrieve_track_features(10,"bronco")
# # print(len(retrieve_track_name(1000)))
# feature_list = retrieve_track_features(10) 
# # print(len(retrieve_track_name(1000)))

In [227]:
song_list2.shape

(1000000, 2)

In [ ]:
def retrieve_track_features(rownumbers,tablename):
    feature_list = []
    conn = create_table(tablename)
    for entry,row in song_list2.head(rownumbers).iterrows():
        f = {}
        artist = row["artist"]
        track = row["track"]
        try:
            spotifyid = get_spotify_id(artist,track)
            features = get_features(spotifyid)
            f = {"artist":artist,"track":track,'danceability':features['danceability'],
                                 'energy':features['energy'],'key':features['key'],
                                 'loudness':features['loudness'],'mode':features['mode'],
                                'speechiness':features['speechiness'],'acousticness':features['acousticness'],
                                 'instrumentalness':features['instrumentalness'],'liveness':features['liveness'],
                                 'valence':features['valence'],'tempo':features['tempo'],'duration_ms':features['duration_ms'],'time_signature':features['time_signature']}
            try:
                insert(conn,f)
            except:
                print("Insert Statement failed")
        except:
            continue


retrieve_track_features(1000000,"song_list2")
# print(len(retrieve_track_name(1000)))
# feature_list = retrieve_track_features(10) 
# print(len(retrieve_track_name(1000)))

Insert Statement failed
Insert Statement failed
Insert Statement failed
Insert Statement failed
Insert Statement failed
Insert Statement failed
Insert Statement failed
Insert Statement failed
Insert Statement failed
Insert Statement failed
Insert Statement failed
Insert Statement failed
